In [41]:
import pulp as plp
from pulp import *

import numpy as np
import pandas as pd

# Problem Data

In [42]:
mine_list = ['Ashley','Bedford','Consol','Dunby','Earlam','Florence','Gaston','Hopt']

mine_capacity = [300, 600, 510, 655, 575, 680, 450, 490] # (mt/yr)

mine_price = [49500, 50000, 61000, 63500, 66500, 71000, 72500, 80000] # ($/mt)

RHS = [1225, 0, 650, 720, 0]

coeff_matrix = np.array([[1,1,1,1,1,1,1,1],
                         [1,1,-1,1,-1,1,-1,-1],
                         [1,0,1,0,0,0,1,1],
                         [0,1,0,1,1,1,0,0],
                         [-0.04,-0.03,-0.01,0.01,0.02,0.03,0.04,0.06]])

In [43]:
n = len(mine_price)  # n = len(mine_list)  n = len(mine_capacity)
m = len(RHS)

set_I = range(1, n+1)
set_J = range(1, m+1)

c = {i: mine_capacity[i-1] for i in set_I}
p = {i: mine_price[i-1] for i in set_I}

b = {j: RHS[j-1] for j in set_J}
a = {(j,i): coeff_matrix[j-1,i-1] for j in set_J for i in set_I}

In [44]:
prob = LpProblem("Coal Supplier Selection")
prob.sense = LpMinimize

In [45]:
x_vars = {i: 
          LpVariable(cat=LpContinuous, lowBound=0, upBound=c[i], name="x_"+mine_list[i-1])
          for i in set_I}

In [46]:
constraints = {j: prob.addConstraint(
    LpConstraint(e=lpSum(a[j,i] * x_vars[i] for i in set_I), sense=0, rhs=b[j], name="constraint_{0}".format(j))) 
               for j in [1]} #different sense: sense = 0, 1: >=, -1: <=

In [47]:
constraints = {j: prob.addConstraint(
    LpConstraint(e=lpSum(a[j,i] * x_vars[i] for i in set_I), sense=1, rhs=b[j], name="constraint_{0}".format(j)))
               for j in [2,5]} #different sense

In [48]:
constraints = {j: prob.addConstraint(
    LpConstraint(e=lpSum(a[j,i] * x_vars[i] for i in set_I), sense=-1, rhs=b[j], name="constraint_{0}".format(j)))
               for j in [3,4]} #different sense

In [49]:
objective = lpSum(x_vars[i] * p[i] for i in set_I)
prob.setObjective(objective) 

In [50]:
prob.solve()

1

In [51]:
# The status of the solution is printed to the screen
print("="*30,"\nSolution Status:", LpStatus[prob.status])

# Results
obj = value(prob.objective)
print("The minimum coal procurement cost for next year is: ${}".format(round(obj,2)))

print("Optimal Soultion:")
for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Constraints")
for c in list(prob.constraints.values()):
    print(c.name, "=", c.value())

Solution Status: Optimal
The minimum coal procurement cost for next year is: $73267500.0
Optimal Soultion:
x_Ashley = 55.0
x_Bedford = 600.0
x_Consol = 0.0
x_Dunby = 20.0
x_Earlam = 100.0
x_Florence = 0.0
x_Gaston = 450.0
x_Hopt = 0.0
Constraints
constraint_1 = 0.0
constraint_2 = 125.0
constraint_5 = 0.0
constraint_3 = -145.0
constraint_4 = 0.0
